# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @gpizzigh

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'asus'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Faculdade\Documents\2º Semestre\Ciência dos Dados\GitHub\CD-Projeto-2\ciencia-dos-dados


In [8]:
dt = pd.read_excel('asus.xlsx',sep = ',')
#dt = dt.Treinamento.str.strip(',')
dt.Treinamento = dt.Treinamento.replace(',','',regex=True)
dt.Treinamento = dt.Treinamento.replace('@','',regex=True)
dt.Treinamento = dt.Treinamento.replace(';','',regex=True)
dt.Treinamento = dt.Treinamento.replace('  ','',regex=True)
dt.Treinamento = dt.Treinamento.replace(':','',regex=True)
dt.Treinamento = dt.Treinamento.replace('"','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\\*','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\*','',regex=True)
dt.Treinamento = dt.Treinamento.replace('_','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\$','',regex=True)
dt.Treinamento = dt.Treinamento.replace('rt','',regex=True)
dt.Treinamento = dt.Treinamento.replace('//','',regex=True)
dt.Treinamento = dt.Treinamento.replace('/','',regex=True)
dt.Treinamento = dt.Treinamento.replace('https','',regex=True)
dt.Treinamento = dt.Treinamento.replace('#','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\(','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\)','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\'','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\?','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\-','',regex=True)
dt.Treinamento = dt.Treinamento.replace('!','',regex=True)
dt.Treinamento = dt.Treinamento.replace('\n','',regex=True) #pode dar erro 
dt

Treinamento Relevancia
0    ahórrate 6330€ comprando asus vg248qe 24 negro...          N
1    meidzzow cara eu iria de asusbenq. mas dizem q...          S
2    que saudade das foto quadradinha asus por favo...          S
3    me acabo de comprar esta maravilla. asus 239vx...          S
4    brigado asus pelo celular acabei de adquiri um...          S
5    se ainda não conheces as novidades asus aprese...          N
6    gleisonjsilva os seguintes dispositivos estão ...          N
7     logggaming inscreva agora soeio placa de víde...          N
8    asus lança os novos notebooks para jogos rog s...          N
9    placa de vídeo asus geforce ... apenas r2099.0...          N
10        cada vez mais decepcionada com a asus asusbr          S
11    logggaming inscreva agora soeio placa de víde...          N
12   gostei de um vídeo youtube t.cojjnall6nzj send...          N
13   submarino comprei com vcs um not asus que de b...          S
14   poátil asus x555qgxo052 1.780.000 en compunota...          N
15   asus revela roteador com a nova geração do wif...          N
16   gostei de um vídeo youtube t.cocmfodkcqsp melh...          N
17   gostei de um vídeo youtube t.cojql6iaqp5p plac...          N
18   smaphone asus zenfone 3 max dourado 16gb t.con...          N
19    carlosjosederez dá uma olhada nessa reclamaçã...          N
20    1781 no caão americanas ou 1902 no boleto  no...          N
21   o vicepresidente da asus eric chen já apresent...          N
22                                saudades do meu asus          S
23   asus lança os novos notebooks para jogos rog s...          N
24    logggaming inscreva agora soeio placa de víde...          N
25   asus ux410uagv059t  poátil de 14 fullhd ips in...          N
26   josluizmonaco1 marcelcampos bora impoar haha. ...          N
27                cbegg keglekongen asus xbox uha  mig          N
28    logggaming inscreva agora soeio placa de víde...          N
29    logggaming inscreva agora soeio placa de víde...          N
..                                                 ...        ...
270  gostei de um vídeo youtube t.cordbaftqol0 asus...          N
271  asus revela roteador com a nova geração do wif...          N
272  os outros notebooks gaming roc que a asus apre...          N
273   logggaming inscreva agora soeio placa de víde...          N
274  eu queria muito que o teclado da asus tivesse ...          S
275  que arrependimento de ter comprado um zenfone ...          S
276  smaphone asus zenfone go dual chip t.co05wrwic...          N
277   logggaming inscreva agora soeio placa de víde...          N
278   logggaming inscreva agora soeio placa de víde...          N
279  eu apaguei mais de 2 mil fotos e ainda ta fala...          S
280  renault renaultkwid renaultscenic renaultcaptu...          N
281                            saudades do meu asus 😥😭          N
282  tive que comprar outro celular o meu outro fic...          S
283  incrível relógio asus zenwatch3 preto ótimo pr...          N
284  asusbr .... pessoal muito cuidado ao comprarem...          S
285  gleisonjsilva os seguintes dispositivos estão ...          N
286   logggaming inscreva agora soeio placa de víde...          N
287    meiobit eis as novidades da asus t.co3hjvuly4dk          N
288  r4ndyg i78gb ramr9 380 4gb e a mobo eh da asus...          N
289   logggaming inscreva agora soeio placa de víde...          N
290  gostei de um vídeo youtube t.coiy9xh1jrqm asus...          N
291   logggaming inscreva agora soeio placa de víde...          N
292  o fone de ouvido da asus eu acho muito superio...          N
293          joanachapz seus inúteis o melhor é o asus          S
294  cara é monstruoso o que a asus tá fazendo com ...          S
295  gostei de um vídeo youtube t.coz6tk048uwp de v...          N
296  aurguidugli e placas da asus perderam um pouco...          S
297  asus revela roteador com a nova geração do wif...          N
298  novos asus zenbook flip 14 flip15 e flip s os ...          N
299  gostei de um

In [9]:
coluna_tre = pd.Series(np.sum(dt.Treinamento + " ").split())
coluna_rel = pd.Series(np.sum(dt.Relevancia + " ").split())

In [10]:
coluna_tre.value_counts()


asus              277
de                152
vídeo              96
e                  72
um                 64
a                  60
o                  60
agora              58
4gb                56
que                54
placa              53
gtx                53
1050               52
ti                 52
128bits            52
gddr5              51
soeio              50
t.…                49
t.cowmx1gxcbfa     49
inscreva           49
inscrevase         49
logggaming         49
youtube            47
do                 46
da                 45
zenfone            43
gostei             40
é                  38
com                36
flip               32
                 ... 
dizem               1
t.coluendukatg      1
submarino           1
qualidade           1
sinto               1
ramgtx              1
apaguei             1
out…                1
vale                1
ícones              1
retweeted           1
experiência         1
t.coags4nyecct      1
100%                1
tenha     

In [11]:
prob = coluna_rel.value_counts('S')

In [12]:
# probabilidade de relevantes do total 
p_S = prob.iloc[1]
p_N = prob.iloc[0]
print('probabilidade de relevantes do total : {0}'.format(p_S))
print('probabilidade de irrelevantes do total : {0}'.format(p_N))

probabilidade de relevantes do total : 0.26
probabilidade de irrelevantes do total : 0.74


In [13]:
dt_rele = dt[dt.Relevancia == 'S']
dt_rele


Treinamento Relevancia
1    meidzzow cara eu iria de asusbenq. mas dizem q...          S
2    que saudade das foto quadradinha asus por favo...          S
3    me acabo de comprar esta maravilla. asus 239vx...          S
4    brigado asus pelo celular acabei de adquiri um...          S
10        cada vez mais decepcionada com a asus asusbr          S
13   submarino comprei com vcs um not asus que de b...          S
22                                saudades do meu asus          S
30   tô ligando pra asus e espero resolver meu prob...          S
31                                   enfim asusbb novo          S
35   adrieldutra asus tão potente q não aguentou a ...          S
44                foda n vender o fone da asus mt foda          S
47   lmjdulce asus omg meu proximo celular a menina...          S
49   vou jogar essa desgraça pela janela.. asus end...          S
60   ainda pra piorar nao roda um dvdaudio e video ...          S
63   depois q a asus atualizou o sistema ficou uma ...          S
71   oraul os ciclos de aparelhos caiu de ano em an...          S
75   gostei de um vídeo youtube t.cor11qvel5if asus...          S
76   ceciliamusical marcelcampos asusbr ñ se trata ...          S
79   anapads considere um asus. o meu é o selfie e ...          S
81   baixei o file manager da asus porque o nativo ...          S
89   durante a ifa 2017 grande evento de tecnologia...          S
92   fico chocado como a câmera desses celulares da...          S
94   hdmaines pp mano asus e sony são os melhores a...          S
96   taniamorenom é bom demais tô com um asus tava ...          S
97   o melhor dos telefones da asus é que ele demor...          S
98    diianar31 só queria saber quem foi o fdp da a...          S
110  ei asus vcs esqueceram de me mandar o quebraca...          S
115  incrível relógio asus zenwatch3 preto ótimo pr...          S
116  marcelcampos quero muito comprar um zenfone to...          S
117     asus marcelcampos nossa q sonho de consumo.😍😍😍          S
..                                                 ...        ...
196  eu odeio muito a asusmeu pc ja era um lixo ai ...          S
203  moscowsyck arrumei o desktop e peguei um notez...          S
206  dá uma olhada nessa reclamação contrano reclam...          S
214  falam q a bateria da asus é a melhor....porra ...          S
215  reclameaqui carlos josé rezende8 min · twitter...          S
217  se um dia me virem comprando algo da asus me i...          S
218  que se foda o iphone 8 não vou comprar mesmo s...          S
219  asusbr até hoje não consegui atendimento da as...          S
222  sem celular de novo e dessa vez morrendo de ód...          S
224  momento de indecisão eu começo a xingar os eng...          S
227  comprei um asus zenfone go até quaafeira ele c...          S
231  thostama mano tem uns asus top uns motorola to...          S
239  guipradella viniisfc caiocnascimento chato do ...          S
240                     seus inúteis o melhor é o asus          S
244                     katrielyds compra um asus sony          S
248          o zen ear da asus é melhor t.corwcjz25vww          S
249  mas nada salva daquele modo de embelezamento d...          S
259  asusbr asusfanaticos não encontro pulseira do ...          S
261  asus tanta saudade do meu celular foram dois d...          S
262            tô voltando pra asus amor da minha vida          S
265  o fone de ouvido da asus estraçalha demais o d...          S
268  ceciliamusical marcelcampos asusbr ñ tenho que...          S
274  eu queria muito que o teclado da asus tivesse ...          S
275  que arrependimento de ter comprado um zenfone ...          S
279  eu apaguei mais de 2 mil fotos e ainda ta fala...          S
282  tive que comprar outro celular o meu outro fic...          S
284  asusbr .... pessoal muito cuidado ao comprarem...          S
293          joanachapz seus inúteis o melhor é o asus          S
294  cara é monstruoso o que a asus tá fazendo com ...          S
296  aurguidugli 

In [14]:
dt_irrele = dt[dt.Relevancia == 'N']
dt_irrele

Treinamento Relevancia
0    ahórrate 6330€ comprando asus vg248qe 24 negro...          N
5    se ainda não conheces as novidades asus aprese...          N
6    gleisonjsilva os seguintes dispositivos estão ...          N
7     logggaming inscreva agora soeio placa de víde...          N
8    asus lança os novos notebooks para jogos rog s...          N
9    placa de vídeo asus geforce ... apenas r2099.0...          N
11    logggaming inscreva agora soeio placa de víde...          N
12   gostei de um vídeo youtube t.cojjnall6nzj send...          N
14   poátil asus x555qgxo052 1.780.000 en compunota...          N
15   asus revela roteador com a nova geração do wif...          N
16   gostei de um vídeo youtube t.cocmfodkcqsp melh...          N
17   gostei de um vídeo youtube t.cojql6iaqp5p plac...          N
18   smaphone asus zenfone 3 max dourado 16gb t.con...          N
19    carlosjosederez dá uma olhada nessa reclamaçã...          N
20    1781 no caão americanas ou 1902 no boleto  no...          N
21   o vicepresidente da asus eric chen já apresent...          N
23   asus lança os novos notebooks para jogos rog s...          N
24    logggaming inscreva agora soeio placa de víde...          N
25   asus ux410uagv059t  poátil de 14 fullhd ips in...          N
26   josluizmonaco1 marcelcampos bora impoar haha. ...          N
27                cbegg keglekongen asus xbox uha  mig          N
28    logggaming inscreva agora soeio placa de víde...          N
29    logggaming inscreva agora soeio placa de víde...          N
32    logggaming inscreva agora soeio placa de víde...          N
33   adicionei um vídeo a uma playlist youtube t.co...          N
34   gostei de um vídeo youtube t.cozq6z2md9d5 touc...          N
36   asus sendo asus zenfone 4 | tkd podcast 02 t.c...          N
37   gostei de um vídeo youtube t.cowvjz6zj03f novo...          N
38   lojakadoo ifa 2017 asus apresenta notebooks fi...          N
39    ivoidhanger postando de novo novamente again ...          N
..                                                 ...        ...
257   gomestreem hdmaines não tenho o que reclamar ...          N
258  incrível relógio asus zenwatch3 preto ótimo pr...          N
260   logggaming inscreva agora soeio placa de víde...          N
263  smaphone asus zenfone go lte 16gb t.coavyjg44e...          N
264  alguém tem o zenfone 3 max ou algum smaphone d...          N
266   logggaming inscreva agora soeio placa de víde...          N
267          isso aqui da uns 100 reais t.covdizhacvan          N
269  10 pulgadas tablet funda cueroultra slim sma c...          N
270  gostei de um vídeo youtube t.cordbaftqol0 asus...          N
271  asus revela roteador com a nova geração do wif...          N
272  os outros notebooks gaming roc que a asus apre...          N
273   logggaming inscreva agora soeio placa de víde...          N
276  smaphone asus zenfone go dual chip t.co05wrwic...          N
277   logggaming inscreva agora soeio placa de víde...          N
278   logggaming inscreva agora soeio placa de víde...          N
280  renault renaultkwid renaultscenic renaultcaptu...          N
281                            saudades do meu asus 😥😭          N
283  incrível relógio asus zenwatch3 preto ótimo pr...          N
285  gleisonjsilva os seguintes dispositivos estão ...          N
286   logggaming inscreva agora soeio placa de víde...          N
287    meiobit eis as novidades da asus t.co3hjvuly4dk          N
288  r4ndyg i78gb ramr9 380 4gb e a mobo eh da asus...          N
289   logggaming inscreva agora soeio placa de víde...          N
290  gostei de um vídeo youtube t.coiy9xh1jrqm asus...          N
291   logggaming inscreva agora soeio placa de víde...          N
292  o fone de ouvido da asus eu acho muito superio...          N
295  gostei de um vídeo youtube t.coz6tk048uwp de v...          N
297  asus revela roteador com a nova geração do wif...          N
298  novos asus zenbook flip 14 flip15 e flip s os ...          N
299  gostei de um

In [15]:
# total de relevantes 
coluna_r = pd.Series(np.sum(dt_rele.Treinamento + " ").split())
coluna_r.value_counts()

asus            69
o               38
de              38
que             28
da              26
a               25
e               24
um              20
não             16
é               15
celular         15
eu              12
do              12
q               11
pra             10
meu              9
tem              9
zenfone          8
asusbr           8
com              8
uma              8
mais             8
comprar          7
se               7
os               6
mas              6
me               6
marcelcampos     6
muito            6
melhor           5
                ..
2.               1
odeio            1
merda            1
comprarem        1
complexabel      1
menina           1
modo             1
s                1
arrumei          1
durante          1
piorar           1
bloqueio         1
carlos           1
falam            1
comprado         1
descarregar      1
maravilla.       1
placas           1
segue            1
meses.           1
amo              1
fdp         

In [22]:
total_rel = len(np.sum(dt_rele.Treinamento + " ").split())
total_rel

1190

In [16]:
#total de irrelevantes
coluna_ir = pd.Series(np.sum(dt_irrele.Treinamento + " ").split())
coluna_ir.value_counts()

asus                     208
de                       114
vídeo                     95
agora                     58
4gb                       56
gtx                       53
placa                     53
128bits                   52
1050                      52
ti                        52
gddr5                     51
soeio                     50
t.cowmx1gxcbfa            49
t.…                       49
inscreva                  49
inscrevase                49
logggaming                49
e                         48
youtube                   46
um                        44
gostei                    39
a                         35
zenfone                   35
do                        34
flip                      30
com                       28
que                       26
os                        25
é                         23
o                         22
                        ... 
bggaming                   1
tô                         1
with                       1
[review]      

In [45]:
nova_coluna = pd.Series(coluna_ir.value_counts())
#nova_coluna[nova_coluna == 208]
#pd.Index(nova_coluna).get_loc()
nova_coluna.str.find('asus')

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

16

In [21]:
total_irre = len(np.sum(dt_irrele.Treinamento + " "))
total_irre

21219

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [27]:
f_re = 'esse backplate é lindo né o que vocês acham'
f_re = f_re.split(' ')
f_re[0]

#P_irre = []
#P_rele = []
for i in f_re:
    f_re[i] = f_re[i]/ 
    

'esse'

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
